In [1]:
from dotenv import load_dotenv
load_dotenv()

from google.adk.agents import Agent
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset, StreamableHTTPConnectionParams

root_agent = Agent(
    model="gemini-2.5-flash",
    name="financial_assistant",
    description="A helpful assistant for user questions.",
    instruction=f"""\
Answer user's questions about personal finance.
You have access to the following tools:
-fetch_net_worth
-fetch_credit_report
-fetch_epf_details
-fetch_mf_transactions
Use them when necessary to answer the user's questions.
Note: when you call a tool for the first time, 
the tool will respond with an authentication link. 
Just give the message returned by the tool as it is to the user. 
Let the user authenticate and then you will receive a mcp session ID, remember this ID.
Make the tool call once again 
after the user has confirmed that they have authenticated.
    """,
    tools=[
        MCPToolset(
            connection_params=StreamableHTTPConnectionParams(
                url="http://localhost:8080/mcp/stream",
            ),
        ),
    ],
)

In [2]:
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "financial_assistant_app"
USER_ID = "2222222222"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- Runner ---
# Key Concept: Runner orchestrates the agent execution loop.
runner = Runner(
    agent=root_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Session created: App='financial_assistant_app', User='2222222222', Session='session_001'
Runner created for agent 'financial_assistant'.


In [3]:
from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")

In [4]:
await call_agent_async("Hi my name is Kirushikesh, what can you do?",
                                    runner=runner,
                                    user_id=USER_ID,
                                    session_id=SESSION_ID)

/Users/kirushikeshdb/Library/Caches/pypoetry/virtualenvs/humsafar-financial-ai-assistant-4inwsc2H-py3.11/lib/python3.11/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:87: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip 


>>> User Query: Hi my name is Kirushikesh, what can you do?
<<< Agent Response: Hi Kirushikesh! I'm your personal financial assistant. I can help you with a variety of tasks related to your finances. Here are some of the things I can do:

* **Fetch your net worth:** I can calculate your total net worth based on your linked bank accounts, investments (mutual funds, Indian and US stocks), EPF balance, and any outstanding debts.
* **Retrieve your credit report:** I can get you a comprehensive credit report, including your credit score, loan details, credit card utilization, and payment history.
* **Fetch your EPF details:** I can provide you with detailed information about your Employee Provident Fund account, such as your balance, contributions, and interest earned.
* **Fetch your mutual fund transactions:** I can retrieve your detailed transaction history for your mutual fund investments.
* **Fetch your bank transactions:** I can get you detailed transaction information from your linke

In [5]:
await call_agent_async("hey finance assistant, do you know my name?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.



>>> User Query: hey finance assistant, do you know my name?
<<< Agent Response: Yes, I do! You told me your name is Kirushikesh.


In [6]:
await call_agent_async("Can you tell me my net worth?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.



>>> User Query: Can you tell me my net worth?


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


<<< Agent Response: I can help you with that. It looks like you need to log in first. Please visit this link to log in: http://localhost:8080/mockWebPage?sessionId=mcp-session-fb131251-23ae-43b2-805d-45ba0e045294

Once you've logged in, please let me know, and I'll fetch your net worth for you.



In [7]:
await call_agent_async("I have authenticated",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.



>>> User Query: I have authenticated


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


<<< Agent Response: Your total net worth is INR 1,360,797.

Here's a breakdown of your assets:
*   **Mutual Funds:** INR 177,605
*   **EPF:** INR 211,111
*   **Indian Securities:** INR 200,642
*   **Savings Accounts:** INR 740,841
*   **US Securities:** INR 30,598


In [8]:
await call_agent_async("Can you summarize my bank statements?",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.



>>> User Query: Can you summarize my bank statements?
<<< Agent Response: I can fetch your detailed bank transactions for all accounts connected to Fi Money. However, I cannot summarize them for you in terms of categorizing expenses or providing spending insights. Would you like to see your detailed bank transactions?


In [9]:
await call_agent_async("yes please",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.



>>> User Query: yes please


auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.


<<< Agent Response: I have retrieved your bank transactions. You can see transactions from HDFC Bank and Union Bank. Each transaction includes the amount, narration, date, type (e.g., CREDIT, DEBIT), mode (e.g., ATM, UPI), and current balance.

Please note that I currently only have access to the last two months of transactions.

Would you like to see the details for a specific bank or are you looking for anything else?



In [24]:
tool_set = MCPToolset(
    connection_params=StreamableHTTPConnectionParams(
        url="http://localhost:8080/mcp/stream",
    ),
)

In [25]:
tools = await tool_set.get_tools()

/Users/kirushikeshdb/Library/Caches/pypoetry/virtualenvs/humsafar-financial-ai-assistant-4inwsc2H-py3.11/lib/python3.11/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:87: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip authentication.Using FunctionTool instead if authentication is not required.
auth_config or auth_config.auth_scheme is missing. Will skip 

In [34]:
await tools[0].run_async(args={}, tool_context=None)

CallToolResult(meta=None, content=[TextContent(type='text', text='{"status": "login_required","login_url": "http://localhost:8080/mockWebPage?sessionId=mcp-session-5d1e2d69-2c9f-4d44-aa8f-80499bbd7738","message": "Needs to login first by going to the login url.\\nShow the login url as clickable link if client supports it. Otherwise display the URL for users to copy and paste into a browser. \\nAsk users to come back and let you know once they are done with login in their browser"}', annotations=None, meta=None)], structuredContent=None, isError=False)

In [35]:
await tools[0].run_async(args={}, tool_context=None)

CallToolResult(meta=None, content=[TextContent(type='text', text='{"schemaDescription":"A list of bank transactions. Each \'txns\' field is a list of data arrays with schema: [transactionAmount, transactionNarration, transactionDate, transactionType (1 for CREDIT, 2 for DEBIT, 3 for OPENING, 4 for INTEREST, 5 for TDS, 6 for INSTALLMENT, 7 for CLOSING and 8 for OTHERS), transactionMode, currentBalance].\\n We currently have have only last two month transaction, older transaction are coming soon...", "bankTransactions":[{"bank":"HDFC Bank", "txns":[["80072", "UPI-MR MOHAMMED ZAHEERUL-8884442498@YBL-IDIB000B041-118513918994-PAYMENT FROM PHONE", "2025-07-09", 2, "ATM", "3676"], ["80558", "UPI-SWIGGY-SWIGGY8@YBL-YESB0YBLUPI-104264587845-PAYMENT FROM PHONE", "2025-07-09", 1, "OTHERS", "-76396"], ["80415", "UPI-CRED-CRED@AXISB-UTIB0000114-030309272514-CREDIT CARD BILL P", "2025-07-09", 1, "CARD_PAYMENT", "-76246"], ["80408", "UPI-ACT BROADBAND-PAYTM-ACTBAN4@PAYTM-PYTM0123456-016839849023-PAYM

In [23]:
await tool_set.close()